In [ ]:
# Importing flask module in the project is mandatory 
# An object of Flask class is our WSGI application. 
from flask import Flask 
import numpy as np 
import pandas as pd
from flask import jsonify
from flask import render_template

import pickle
# Flask constructor takes the name of  
# current module (__name__) as argument. 
app = Flask(__name__) 
  
# The route() function of the Flask class is a decorator,  
# which tells the application which URL should call  
# the associated function. 
@app.route('/')
def home():
    return render_template('index.html')
# /api/bedrooms=4&bathrooms=3&area=Carmichael

@app.route('/api/bedrooms=<bedrooms>&bathrooms=<bathrooms>&area=<area>') 
def rent_estimate(bedrooms,bathrooms,area):
    no_bathrooms = bathrooms
    no_bedrooms = bedrooms
    area = area
    
    model = pickle.load(open('pickle_model.pkl','rb'))
    encoder = pickle.load(open('encoder.pkl','rb'))
    minmax = pickle.load(open('minmax.pkl','rb'))
    d = {'Bedrooms': [no_bedrooms], 'Bathrooms': [no_bathrooms]}
    df = pd.DataFrame(data=d)
    citytransform = encoder.transform([area])
    citytransform = pd.DataFrame(citytransform)
    df = pd.concat([df, citytransform], axis=1)
    ans = model.predict(df).reshape(-1,1)
    rent_estimate = minmax.inverse_transform(ans)[0][0]
    response = {
        'Bedrooms':no_bedrooms,
        'Bathrooms':no_bathrooms,
        'Area':area,
        'Estimated Rent':rent_estimate
    }
    json_resp = jsonify(response)
    json_resp.status_code = 200
    print(json_resp)
    return json_resp

# main driver function 
if __name__ == '__main__': 
  
    # run() method of Flask class runs the application  
    # on the local development server. 
    app.run() 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jan/2020 09:34:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2020 09:34:22] "GET /api/bedrooms=4&bathrooms=3&area=Carmichael HTTP/1.1" 200 -


<Response 88 bytes [200 OK]>
